# Imports

In [1]:
import subprocess
import base64
import os
import requests
from datetime import datetime, timedelta
import xml.etree.ElementTree as ET
import json

from bill_request import Bill_HttpRequest

# Check si ".crt" y ".pem" son validos para AFIP tranajando en testing

In [2]:
def is_token_valid(ta_path="TA.xml"):
    try:
        tree = ET.parse(ta_path)
        root = tree.getroot()
        expiration = root.find(".//expirationTime").text
        expiration_dt = datetime.strptime(expiration[:19], "%Y-%m-%dT%H:%M:%S")
        return datetime.now() < expiration_dt
    except Exception as e:
        print("❌ Error leyendo TA.xml:", e)
        return False

def obtener_nuevo_token_y_sign(cert_path, key_path, service="wsfe", wsaa_url="https://wsaahomo.afip.gov.ar/ws/services/LoginCms"):
    """
    Desde certificado y clave privada, obtener TA.xml (token de acceso) para el servicio wsfe (web service de facturación electrónica).

    Si el WSAA responde con TA.xml significa que:
    - El .crt y .key coinciden.
    - El .csr fue generado con el CUIT correcto en el CN.
    - El certificado fue aceptado por AFIP para el servicio wsfe.
    - Las fechas de generación y expiración en el XML están dentro del rango permitido.

    Desde TA.xml obtener:
    - token
    - sign
    - expirationTime
    - ta_xml (archivo completo)
    """
    # === Paths temporales ===
    tra_path = "/tmp/TRA.xml"
    cms_path = "/tmp/TRA_firma.cms"

    # === 1. Crear el TRA.xml ===
    generation_time = (datetime.now() - timedelta(minutes=10)).strftime("%Y-%m-%dT%H:%M:%S")
    expiration_time = (datetime.now() + timedelta(minutes=10)).strftime("%Y-%m-%dT%H:%M:%S")
    tra_xml = f"""<?xml version="1.0" encoding="UTF-8"?>
<loginTicketRequest version="1.0">
  <header>
    <uniqueId>{int(datetime.now().timestamp())}</uniqueId>
    <generationTime>{generation_time}</generationTime>
    <expirationTime>{expiration_time}</expirationTime>
  </header>
  <service>{service}</service>
</loginTicketRequest>
"""
    with open(tra_path, "w") as f:
        f.write(tra_xml)

    # === 2. Firmar el TRA.xml con OpenSSL ===
    subprocess.run([
        "openssl", "smime", "-sign",
        "-signer", cert_path,
        "-inkey", key_path,
        "-in", tra_path,
        "-out", cms_path,
        "-outform", "DER",
        "-nodetach"
    ], check=True)

    # === 3. Leer CMS y codificar en base64 ===
    with open(cms_path, "rb") as f:
        cms_data = f.read()
        cms_encoded = base64.b64encode(cms_data).decode()

    # === 4. Crear el request SOAP para WSAA ===
    soap_request = f"""<?xml version="1.0" encoding="UTF-8"?>
<soapenv:Envelope xmlns:soapenv="http://schemas.xmlsoap.org/soap/envelope/"
                  xmlns:ser="http://wsaa.view.sua.dvadac.desein.afip.gov">
  <soapenv:Header/>
  <soapenv:Body>
    <ser:loginCms>
      <request>{cms_encoded}</request>
    </ser:loginCms>
  </soapenv:Body>
</soapenv:Envelope>
"""
    headers = {
        "Content-Type": "text/xml; charset=utf-8",
        "SOAPAction": "loginCms"
    }

    # === 5. Enviar request al WSAA ===
    response = requests.post(wsaa_url, data=soap_request.encode("utf-8"), headers=headers)

    if response.status_code != 200:
        print("❌ Error en la conexión o autenticación:")
        print(response.text[:1000])
        return None

    # === 6. Parsear respuesta y extraer TA.xml ===
    soap_tree = ET.fromstring(response.text)
    ns = {"soap": "http://schemas.xmlsoap.org/soap/envelope/"}
    login_return_encoded = soap_tree.find(".//soap:Body/*/*", namespaces=ns).text
    ta_xml_str = login_return_encoded

    # === 7. Guardar y parsear TA.xml ===
    with open("TA.xml", "w", encoding="utf-8") as f:
        f.write(ta_xml_str)

    ta_tree = ET.fromstring(ta_xml_str)
    token = ta_tree.find(".//token").text
    sign = ta_tree.find(".//sign").text
    expiration_time = ta_tree.find(".//expirationTime").text

    print("✅ TOKEN y SIGN extraídos correctamente.")

    return {
        "token": token,
        "sign": sign,
        "expiration": expiration_time,
        "ta_xml": ta_xml_str
    }

def obtener_token_sign(cert_path, key_path, service="wsfe", wsaa_url="https://wsaahomo.afip.gov.ar/ws/services/LoginCms"):
    """
    Obtener el token y sign para el servicio wsfe de AFIP.
    Si el token es válido, lo devuelve. Si no, obtiene uno nuevo.
    """
    if is_token_valid():
        print("✅ TOKEN y SIGN válidos.")
        with open("TA.xml", "r") as f:
            ta_xml_str = f.read()
        ta_tree = ET.fromstring(ta_xml_str)
        token = ta_tree.find(".//token").text
        sign = ta_tree.find(".//sign").text
        expiration_time = ta_tree.find(".//expirationTime").text
        return {
            "token": token,
            "sign": sign,
            "expiration": expiration_time,
            "ta_xml": ta_xml_str
        }
    else:
        print("TOKEN y SIGN vencidos. Obteniendo uno nuevo...")
        return obtener_nuevo_token_y_sign(cert_path, key_path, service, wsaa_url)

In [4]:
# === CONFIGURACIÓN ===
CERT_PATH = r"C:\Users\shern\Desktop\Innoview\Facturador-AFIP\Certificados\FacturadorTest\Certificado_FacturadorTest.crt"  # ← Reemplazar con tu ruta real
KEY_PATH = r"C:\Users\shern\Desktop\Innoview\Facturador-AFIP\Certificados\FacturadorTest\private_key_20412219652_2025-05-15-202213.pem"  # ← Reemplazar con tu ruta real
TRA_PATH = "/tmp/TRA.xml"
CMS_PATH = "/tmp/TRA_firma.cms"
SERVICE = "wsfe"
WSAA_URL = "https://wsaahomo.afip.gov.ar/ws/services/LoginCms"

result = obtener_token_sign(CERT_PATH, KEY_PATH, SERVICE, WSAA_URL)

token = result["token"]
sign = result["sign"]
expiration = result["expiration"]

print("Token:", result["token"])
print("Sign:", result["sign"])
print("Expiration:", result["expiration"])
print("TA XML:", result["ta_xml"])  

✅ TOKEN y SIGN válidos.
Token: PD94bWwgdmVyc2lvbj0iMS4wIiBlbmNvZGluZz0iVVRGLTgiIHN0YW5kYWxvbmU9InllcyI/Pgo8c3NvIHZlcnNpb249IjIuMCI+CiAgICA8aWQgc3JjPSJDTj13c2FhaG9tbywgTz1BRklQLCBDPUFSLCBTRVJJQUxOVU1CRVI9Q1VJVCAzMzY5MzQ1MDIzOSIgZHN0PSJDTj13c2ZlLCBPPUFGSVAsIEM9QVIiIHVuaXF1ZV9pZD0iMzE2MDc4MDQwNyIgZ2VuX3RpbWU9IjE3NDgwODkxOTciIGV4cF90aW1lPSIxNzQ4MTMyNDU3Ii8+CiAgICA8b3BlcmF0aW9uIHR5cGU9ImxvZ2luIiB2YWx1ZT0iZ3JhbnRlZCI+CiAgICAgICAgPGxvZ2luIGVudGl0eT0iMzM2OTM0NTAyMzkiIHNlcnZpY2U9IndzZmUiIHVpZD0iU0VSSUFMTlVNQkVSPUNVSVQgMjA0MTIyMTk2NTIsIENOPWZhY3R1cmFkb3J0ZXN0IiBhdXRobWV0aG9kPSJjbXMiIHJlZ21ldGhvZD0iMjIiPgogICAgICAgICAgICA8cmVsYXRpb25zPgogICAgICAgICAgICAgICAgPHJlbGF0aW9uIGtleT0iMjA0MTIyMTk2NTIiIHJlbHR5cGU9IjQiLz4KICAgICAgICAgICAgPC9yZWxhdGlvbnM+CiAgICAgICAgPC9sb2dpbj4KICAgIDwvb3BlcmF0aW9uPgo8L3Nzbz4K
Sign: RJoHNIhWH7QjkRGrRSAooMNkurhsWxo8xgq8EoGIPWI8hJ+RgMfNz+ZgfPsBeZgnJDJ09xxL5Aeft5PLN6ETTQvBwe0tjhp0toNLS3SYVb60UX9FJvalp4aqzAf1YYdj9/bV7rtce5vi7bD7FwtPLXae6gfZCJQViB5WEriGga8=
Expiration: 2025-05-2

# Generar factura

In [5]:
def consultar_ultimo_numero_autorizado(token, sign, cuit, pto_vta, cbte_tipo):
        soap_request = f"""<?xml version="1.0" encoding="UTF-8"?>
        <soapenv:Envelope xmlns:soapenv="http://schemas.xmlsoap.org/soap/envelope/"
            xmlns:ar="http://ar.gov.afip.dif.FEV1/">
          <soapenv:Header/>
          <soapenv:Body>
            <ar:FECompUltimoAutorizado>
              <ar:Auth>
                <ar:Token>{token}</ar:Token>
                <ar:Sign>{sign}</ar:Sign>
                <ar:Cuit>{cuit}</ar:Cuit>
              </ar:Auth>
              <ar:PtoVta>{pto_vta}</ar:PtoVta>
              <ar:CbteTipo>{cbte_tipo}</ar:CbteTipo>
            </ar:FECompUltimoAutorizado>
          </soapenv:Body>
        </soapenv:Envelope>
        """

        headers = {
            "Content-Type": "text/xml; charset=utf-8",
            "SOAPAction": "http://ar.gov.afip.dif.FEV1/FECompUltimoAutorizado"
        }

        response = requests.post(
            "https://wswhomo.afip.gov.ar/wsfev1/service.asmx",
            data=soap_request.encode("utf-8"),
            headers=headers
        )

        ns = {"soap": "http://schemas.xmlsoap.org/soap/envelope/", "ar": "http://ar.gov.afip.dif.FEV1/"}
        tree = ET.fromstring(response.text)
        nodo = tree.find(".//ar:FECompUltimoAutorizadoResult/ar:CbteNro", namespaces=ns)

        if nodo is None:
            raise Exception("No se pudo obtener el número de comprobante autorizado.")
        return int(nodo.text)

def obtener_puntos_venta(token: str, sign: str, cuit_emisor: int):
    """
    Consulta los puntos de venta habilitados para el emisor y los comprobantes permitidos en cada uno.
    """
    url = "https://servicios1.afip.gov.ar/wsfev1/service.asmx" # Cambiar a prod si es necesario

    headers = {
        "Content-Type": "text/xml; charset=utf-8",
        "SOAPAction": "http://ar.gov.afip.dif.FEV1/FEParamGetPtosVenta"
    }

    soap_request = f"""<?xml version="1.0" encoding="UTF-8"?>
    <soapenv:Envelope xmlns:soapenv="http://schemas.xmlsoap.org/soap/envelope/"
                      xmlns:ar="http://ar.gov.afip.dif.FEV1/">
      <soapenv:Header/>
      <soapenv:Body>
        <ar:FEParamGetPtosVenta>
          <ar:Auth>
            <ar:Token>{token}</ar:Token>
            <ar:Sign>{sign}</ar:Sign>
            <ar:Cuit>{cuit_emisor}</ar:Cuit>
          </ar:Auth>
        </ar:FEParamGetPtosVenta>
      </soapenv:Body>
    </soapenv:Envelope>
    """

    response = requests.post(url, data=soap_request, headers=headers)

    if response.status_code != 200:
        raise Exception(f"Error en la solicitud SOAP: {response.status_code} - {response.text}")

    # Parsear XML
    root = ET.fromstring(response.content)
    namespace = {'ar': 'http://ar.gov.afip.dif.FEV1/'}

    puntos_venta = []
    for result in root.findall('.//ar:ResultGet', namespace):
        pto_vta = result.find('ar:PtoVta', namespace).text
        cbte_tipo = result.find('ar:CbteTipo', namespace).text
        puntos_venta.append({
            'punto_venta': int(pto_vta),
            'tipo_comprobante': int(cbte_tipo)
        })

    return puntos_venta

def emitir_factura_afip(cuit_emisor, punto_venta, factura_tipo, token, sign,
                        importe_total, importe_neto, importe_iva):
    # === 1. Preparar datos de la factura ===
    cbte_nro_anterior = consultar_ultimo_numero_autorizado(token, sign, cuit_emisor, punto_venta, factura_tipo)
    print("Ultimo comprobante emitido:" , cbte_nro_anterior)
    cbte_nro_nuevo = cbte_nro_anterior + 1
    fecha_cbte = datetime.now().strftime("%Y%m%d")

    # === 2. Crear SOAP request para FECAESolicitar ===
    bill = Bill_HttpRequest(
                            CUIT_emisor=cuit_emisor,
                            token=token,
                            sign=sign,
                            punto_venta=punto_venta,
                            cantidad_comprobantes=1,
                            metodo_pago=1,
                            importe_total=importe_total,
                            importe_neto=importe_neto,
                            importe_total_concepto=0,  # ¡Este parámetro es obligatorio!
                            nro_comprobante=cbte_nro_nuevo,
                            fecha_comprobante=fecha_cbte
                        )   

    soap_request = bill.get_request()

    print(f"soap_request:{soap_request}")
    
    headers = {
        "Content-Type": "text/xml; charset=utf-8",
        "SOAPAction": "http://ar.gov.afip.dif.FEV1/FECAESolicitar"
    }

    response = requests.post(
        "https://wswhomo.afip.gov.ar/wsfev1/service.asmx",
        data=soap_request.encode("utf-8"),
        headers=headers
    )

    # === 3. Parsear respuesta ===
    ns = {
        "soap": "http://schemas.xmlsoap.org/soap/envelope/",
        "ar": "http://ar.gov.afip.dif.FEV1/"
    }

    tree = ET.fromstring(response.text)
    result_node = tree.find(".//ar:FECAEDetResponse", namespaces=ns)

    if result_node is None:
        print("❌ No se pudo encontrar el nodo 'FECAEDetResponse'. Respuesta completa:")
        print(response.text)
        return None

    resultado = result_node.find("ar:Resultado", namespaces=ns).text

    if resultado == "A":
        cae = result_node.find("ar:CAE", namespaces=ns).text
        cae_vto = result_node.find("ar:CAEFchVto", namespaces=ns).text

        qr_data = {
            "ver": 1,
            "fecha": f"{fecha_cbte[:4]}-{fecha_cbte[4:6]}-{fecha_cbte[6:]}",
            "cuit": int(cuit_emisor),
            "ptoVta": punto_venta,
            "tipoCmp": factura_tipo,
            "nroCmp": cbte_nro_nuevo,
            "importe": importe_total,
            "moneda": "PES",
            "ctz": 1.0,
            "tipoDocRec": 99,
            "nroDocRec": 0,
            "tipoCodAut": "E",
            "codAut": int(cae)
        }

        json_qr = base64.urlsafe_b64encode(json.dumps(qr_data).encode()).decode()
        url_qr = f"https://www.afip.gob.ar/fe/qr/?p={json_qr}"
        return cae, cae_vto, url_qr
    else:
        print("❌ Error al generar factura.")
        print(response.text)
        return None

In [8]:
"""

"""
cae, cae_vto, qr = emitir_factura_afip(
    cuit_emisor="20412219652",
    punto_venta=1,
    factura_tipo=11,  # Factura C
    token=token,
    sign=sign,
    importe_total=1000,
    importe_neto=1000.00,
    importe_iva=0.00,
)

print("CAE:", cae)
print("Vencimiento CAE:", cae_vto)
print("QR:", qr)

Ultimo comprobante emitido: 15
soap_request:<?xml version="1.0" encoding="UTF-8"?>
                        <soapenv:Envelope xmlns:soapenv="http://schemas.xmlsoap.org/soap/envelope/"
                                        xmlns:ar="http://ar.gov.afip.dif.FEV1/">
                        <soapenv:Header/>
                        <soapenv:Body>
                                <ar:FECAESolicitar>
                                        <ar:Auth>
                                                <ar:Token>PD94bWwgdmVyc2lvbj0iMS4wIiBlbmNvZGluZz0iVVRGLTgiIHN0YW5kYWxvbmU9InllcyI/Pgo8c3NvIHZlcnNpb249IjIuMCI+CiAgICA8aWQgc3JjPSJDTj13c2FhaG9tbywgTz1BRklQLCBDPUFSLCBTRVJJQUxOVU1CRVI9Q1VJVCAzMzY5MzQ1MDIzOSIgZHN0PSJDTj13c2ZlLCBPPUFGSVAsIEM9QVIiIHVuaXF1ZV9pZD0iMzE2MDc4MDQwNyIgZ2VuX3RpbWU9IjE3NDgwODkxOTciIGV4cF90aW1lPSIxNzQ4MTMyNDU3Ii8+CiAgICA8b3BlcmF0aW9uIHR5cGU9ImxvZ2luIiB2YWx1ZT0iZ3JhbnRlZCI+CiAgICAgICAgPGxvZ2luIGVudGl0eT0iMzM2OTM0NTAyMzkiIHNlcnZpY2U9IndzZmUiIHVpZD0iU0VSSUFMTlVNQkVSPUNVSVQgMjA0MTIyMTk